In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Nonlinear Feasibility Problem

Find a point that satisfies several nonlinear constraints, taken from [this MATLAB example](https://www.mathworks.com/help/optim/ug/solve-feasibility-problem.html)

## Problem Description

$$(y+x^2)^2+0.1y^2\leq1$$
$$y\leq\exp(-x)-3$$
$$y\leq x-4$$ 
Does any point $(x,y)$ satisfy all of the constraints?

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [2]:
import time
import torch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [3]:
device = torch.device("cpu")

# variables and corresponding dimensions.
var_in = {"x": [1, 1], "y": [1, 1]}


def comb_fn(X_struct):
    x = X_struct.x
    y = X_struct.y
    # constant objective function
    f = 0 * x + 0 * y

    # inequality constraint
    ci = pygransoStruct()
    ci.c1 = (y + x**2) ** 2 + 0.1 * y**2 - 1
    ci.c2 = y - torch.exp(-x) - 3
    ci.c3 = y - x + 4

    # equality constraint
    ce = None

    return [f, ci, ce]

## User Options
Specify user-defined options for PyGRANSO 

In [4]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 1
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
opts.x0 = 0 * torch.ones((2, 1)).to(device=device, dtype=torch.double)

## Main Algorithm

In [5]:
start = time.time()
cpu_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(
    "PyGRANSO finds a feaible point:(%f,%f)"
    % (cpu_soln.final.x[0], cpu_soln.final.x[1])
)



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## CUDA Computation

In [6]:
device = torch.device("cuda")

# variables and corresponding dimensions.
var_in = {"x": [1, 1], "y": [1, 1]}


def comb_fn(X_struct):
    x = X_struct.x
    y = X_struct.y
    # constant objective function
    f = 0 * x + 0 * y

    # inequality constraint
    ci = pygransoStruct()
    ci.c1 = (y + x**2) ** 2 + 0.1 * y**2 - 1
    ci.c2 = y - torch.exp(-x) - 3
    ci.c3 = y - x + 4

    # equality constraint
    ce = None

    return [f, ci, ce]

In [7]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 1
# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
opts.x0 = 0 * torch.ones((2, 1)).to(device=device, dtype=torch.double)

In [8]:
start = time.time()
cuda_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(
    "PyGRANSO finds a feaible point:(%f,%f)"
    % (cuda_soln.final.x[0], cuda_soln.final.x[1])
)



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## Testing

In [9]:
print(
    cpu_soln.final.x[0],
    cuda_soln.final.x[0].cpu(),
)

print(
    cpu_soln.final.x[1],
    cuda_soln.final.x[1].cpu(),
)

tensor([1.6887], dtype=torch.float64) tensor([1.6548], dtype=torch.float64)
tensor([-2.4866], dtype=torch.float64) tensor([-2.3452], dtype=torch.float64)


In [10]:
torch.testing.assert_close(
    cpu_soln.final.x[0],
    cuda_soln.final.x[0].cpu(),
    atol=1e-6,
    rtol=1e-6,
)

torch.testing.assert_close(
    cpu_soln.final.x[1],
    cuda_soln.final.x[1].cpu(),
    atol=1e-6,
    rtol=1e-6,
)

AssertionError: Tensor-likes are not close!

Mismatched elements: 1 / 1 (100.0%)
Greatest absolute difference: 0.033896733304266125 at index (0,) (up to 1e-06 allowed)
Greatest relative difference: 0.020484333839166338 at index (0,) (up to 1e-06 allowed)